In [1]:
import os
import sys

import semantic_kernel as sk

In [2]:

kernel = sk.Kernel()

In [3]:
import semantic_kernel.connectors.ai.open_ai as skaoai

In [4]:
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
kernel.add_chat_service("azure_chat_competion_service", skaoai.AzureChatCompletion(deployment,endpoint,api_key=api_key,api_version = "2023-12-01-preview"))

Kernel(plugins=KernelPluginCollection(plugins={}), prompt_template_engine=PromptTemplateEngine(), memory=NullMemory(), text_completion_services={'azure_chat_competion_service': <function Kernel.add_text_completion_service.<locals>.<lambda> at 0x125d7b7f0>}, chat_services={'azure_chat_competion_service': <function Kernel.add_chat_service.<locals>.<lambda> at 0x125d7a8c0>}, text_embedding_generation_services={}, default_text_completion_service='azure_chat_competion_service', default_chat_service='azure_chat_competion_service', default_text_embedding_generation_service=None, retry_mechanism=PassThroughWithoutRetry(), function_invoking_handlers={}, function_invoked_handlers={})

In [5]:
from semantic_kernel.planning.basic_planner import BasicPlanner
planner = BasicPlanner()

In [6]:
base_skills_directory = '../../../plugins'

In [7]:
import APIPlugin.CustomPlugin as custom_plugin

In [8]:
#custom_plugin = kernel.import_native_skill_from_directory(base_skills_directory , "APIPlugin")
custom_plugin = kernel.import_plugin(custom_plugin.CustomPlugin(), plugin_name="CustomPlugin")  #.import_native_skill_from_directory(base_plugin , "APIPlugin")
writer_plugin = kernel.import_semantic_plugin_from_directory(base_skills_directory, "WriterPlugin")
email_plugin = kernel.import_semantic_plugin_from_directory(base_skills_directory, "EmailPlugin")
translate_plugin = kernel.import_semantic_plugin_from_directory(base_skills_directory, "TranslatePlugin")

In [17]:
ask = """
Write email about  travellings tips based on  getting current weather in Guangzhou and translate it to Chinese
"""
original_plan = await planner.create_plan(ask, kernel)

In [18]:
print(original_plan.generated_plan)

{
    "input": "travelling tips for Guangzhou",
    "subtasks": [
        {"function": "CustomPlugin.WeatherFunction", "args": {"city": "Guangzhou"}},
        {"function": "WriterPlugin.Tips"},
        {"function": "EmailPlugin.WeatherMail"},
        {"function": "TranslatePlugin.MultiLanguage", "args": {"language": "Chinese"}}
    ]
}


In [19]:
results = await planner.execute_plan(original_plan, kernel)

Variable `$language` not found


In [20]:
results

'主题：我们旅行的重要天气和穿着提示\n\n亲爱的同事们，\n\n希望这封邮件找到你们时一切都好。我们即将开始我们的旅程，我想分享一些重要的提示，以确保我们在旅行中的舒适和安全，特别是考虑到我们可能遇到的炎热天气条件。\n\n预计温度将在30摄氏度左右，因此我们穿着得体以保持凉爽和舒适至关重要。以下是一些提示：\n\n1. 穿浅色衣服：浅色反射阳光和热量，可以帮助我们保持凉爽。\n2. 选择轻质材料：选择像棉花和亚麻这样透气的面料，可以帮助保持舒适的体温。\n3. 穿宽松的衣服：宽松的衣服可以让空气流通，帮助我们保持凉爽。\n4. 保护自己免受阳光的伤害：不要忘记带上太阳镜，帽子和防晒霜，以保护你的皮肤免受阳光的伤害。\n5. 保持水分：喝足够的水以避免脱水。\n\n请在为我们的旅行打包时记住这些提示。在旅行期间，我们所有人都要照顾好自己和他人。\n\n另外，我想提醒大家随时注意旅行条件。让我们互相通报可能出现的任何变化或问题。\n\n祝你们旅途平安，期待我们的共同旅程。\n\n顺祝商祺，\n\n[你的名字]'